# Laborator 10

În cadrul laboratorului, prin rezolvarea [problemei damelor](https://ro.wikipedia.org/wiki/Problema_damelor) (varianta generală), vom studia reprezentarea de tip permutație și operatorii de mutație și încucișare compatibili.


Încă odată, începem prin instalarea DEAP, folosid comanda `pip install`:

In [ ]:
!pip install deap

     |████████████████████████████████| 163kB 5.3MB/s 


Importarea componentelor și librăriilor principale:

In [ ]:
from deap import algorithms
from deap import base
from deap import creator
from deap import tools

import numpy as np
import random
import array

# Probleme celor N dame

Conform [descrierii](https://ro.wikipedia.org/wiki/Problema_damelor):

  *Problema damelor (sau problema reginelor) tratează plasarea a opt regine de șah pe o tablă de șah astfel încât să nu existe două regine care se amenință reciproc. Astfel, se caută o soluție astfel încât nicio pereche de două regine să nu fie pe același rând, pe aceeași coloană, sau pe aceeași diagonală.*

Dacă fiecărei regine îi este alocată o coloană fixă, printr-o reprezentare de tip permutație putem determina care este singura regină pe fiecare linie.

În funcția "fitness" vom număra câte perechi de dame se atacă reciproc pe diagonală. Cu ajutorul algritmului evolutiv, vom căuta configurația care minimează acest număr al conflicteelor.

In [ ]:
def checkConflicts(individual):
  n = len(individual)

  left_diagonal = np.zeros(2*n-1)
  right_diagonal = np.zeros(2*n-1)
    
  # Calculam numarul de regine pe fiecare diagonala
  for i in range(n):
      left_diagonal[i+individual[i]] += 1
      right_diagonal[n-1-i+individual[i]] += 1
    
  # Calculam numarul de conflicte pe fiecare diagonala
  numConflicts = 0
  for i in range(2*n-1):
      if left_diagonal[i] > 1:
            numConflicts += left_diagonal[i] - 1 # o piesa 
      if right_diagonal[i] > 1:
            numConflicts += right_diagonal[i] - 1
  return numConflicts, 
  #pentru ca va fi folosita ca functie fitness, rezultatul returnat trebuie sa fie iterabil 
  #chiar daca avem o singura valoare, punem si o virgula

Definim o funcție pentru afișarea (pe tablă) a unei configurații codificate. 

In [ ]:
def displayBoard(individual):
  n = len(individual)
  board = [['.'] * n for _ in range(n)] 
  
  for index in range(0, n):
    row = index
    column = individual[index] 
    board[column][row] = 'Q'
  
  for i in reversed(range(len(board))):
    print(' '.join(board[i]))

Testare:

In [ ]:
ind = [0,1,2,3,4,5,6,7]
displayBoard(ind)
print(checkConflicts(ind))

. . . . . . . Q
. . . . . . Q .
. . . . . Q . .
. . . . Q . . .
. . . Q . . . .
. . Q . . . . .
. Q . . . . . .
Q . . . . . . .
(7.0,)


In [ ]:
ind = [2,4,1,7,0,6,3,5]
displayBoard(ind)
print(checkConflicts(ind))

. . . Q . . . .
. . . . . Q . .
. . . . . . . Q
. Q . . . . . .
. . . . . . Q .
Q . . . . . . .
. . Q . . . . .
. . . . Q . . .
(0,)


In [ ]:
for i in range(0, 5):
  ind = np.random.permutation(8)
  displayBoard(ind)
  print(checkConflicts(ind))

. . . . . . . Q
. . . Q . . . .
. . . . Q . . .
. . . . . . Q .
Q . . . . . . .
. . . . . Q . .
. Q . . . . . .
. . Q . . . . .
(5.0,)
. . . . . . Q .
. Q . . . . . .
. . . . . Q . .
. . Q . . . . .
. . . . Q . . .
Q . . . . . . .
. . . . . . . Q
. . . Q . . . .
(2.0,)
. . . Q . . . .
. Q . . . . . .
. . . . . . . Q
. . Q . . . . .
. . . . . . Q .
Q . . . . . . .
. . . . . Q . .
. . . . Q . . .
(3.0,)
. . . . . . Q .
. . . . Q . . .
. . . . . Q . .
. Q . . . . . .
. . . Q . . . .
Q . . . . . . .
. . Q . . . . .
. . . . . . . Q
(3.0,)
. . . . Q . . .
. . . . . Q . .
. . . Q . . . .
Q . . . . . . .
. . Q . . . . .
. . . . . . . Q
. Q . . . . . .
. . . . . . Q .
(2.0,)


Definim lungimea "cromosomului", a soluției. În formularea/reprezentarea folosită, fiecare piesă are o coordonată prestabilită, trebuie să codicăm doar a doua coordonată. 

In [ ]:
NB_QUEENS = 8 # numarul pieselor
IND_SIZE = NB_QUEENS * 1 # doar o singura coordonata

Definim reprezenatrea soluțiilor, strategia de selecție și funcția fitness.

In [ ]:
# dorim să minimizăm numărul conflictelor
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

# reprezenatre - un șir de numere (ce formeaza o permutatie)
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox = base.Toolbox()
toolbox.register("permutation", random.sample, range(IND_SIZE), IND_SIZE)

toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.permutation)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", checkConflicts)

# folosim operatori de variatie adecvate reprezentarii tip permutatie
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=2.0/IND_SIZE)

# selectie de tip turnir
toolbox.register("select", tools.selTournament, tournsize=4)

/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Cu ajutorul modului `Statistics` putem genera (și afișa) statisticile aferente procesului de optimizare.

In [ ]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

# reținem cele mai bune 10 soluții
hof = tools.HallOfFame(10)

In [ ]:
pop = toolbox.population(n=100)

# de data asta folosim o versiune cu selectie de tip generational (eaSimple)
pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=100, stats=stats,
                        halloffame=hof, verbose=True)

gen	nevals	avg 	std    	min	max
0  	100   	4.05	1.43091	1  	8  
1  	52    	3.04	1.22409	0  	6  
2  	60    	3.01	1.46625	0  	7  
3  	50    	2.37	1.55342	0  	7  
4  	67    	2.39	1.47577	0  	5  
5  	63    	2.32	1.51578	0  	6  
6  	55    	1.97	1.4997 	0  	6  
7  	72    	1.72	1.57531	0  	6  
8  	55    	1.23	1.68437	0  	6  
9  	52    	0.64	1.36763	0  	5  
10 	51    	0.38	1.08425	0  	5  
11 	68    	0.68	1.34818	0  	5  
12 	59    	0.97	1.87326	0  	9  
13 	76    	0.58	1.4641 	0  	7  
14 	51    	0.64	1.41082	0  	6  
15 	53    	0.49	1.20412	0  	5  
16 	62    	0.62	1.26317	0  	4  
17 	68    	0.59	1.30457	0  	5  
18 	64    	0.43	1.13362	0  	5  
19 	55    	0.5 	1.22066	0  	5  
20 	66    	0.58	1.42955	0  	6  
21 	54    	0.64	1.41082	0  	6  
22 	69    	0.52	1.34521	0  	7  
23 	72    	0.57	1.36569	0  	6  
24 	56    	0.5 	1.27671	0  	7  
25 	61    	0.44	1.12534	0  	5  
26 	68    	0.89	1.73144	0  	6  
27 	74    	0.58	1.25841	0  	5  
28 	54    	0.62	1.30215	0  	6  
29 	51    	0.52	1.19566	0  	5  
30 	58  

Afișsarea celor mai bune soluții:

In [ ]:
for h in hof:
  displayBoard(h)
  print('Above configuration has the fitness: {}'.format(h.fitness))

. . . . . Q . .
. Q . . . . . .
. . . . . . Q .
Q . . . . . . .
. . Q . . . . .
. . . . Q . . .
. . . . . . . Q
. . . Q . . . .
Above configuration has the fitness: (0.0,)
. . . . Q . . .
. Q . . . . . .
. . . . . . . Q
Q . . . . . . .
. . . Q . . . .
. . . . . . Q .
. . Q . . . . .
. . . . . Q . .
Above configuration has the fitness: (0.0,)
. . . . Q . . .
. Q . . . . . .
. . . . . Q . .
Q . . . . . . .
. . . . . . Q .
. . . Q . . . .
. . . . . . . Q
. . Q . . . . .
Above configuration has the fitness: (0.0,)
. . Q . . . . .
Q . . . . . . .
. . . . . . Q .
. Q . . . . . .
. . . Q . . . .
. . . . . Q . .
. . . . . . . Q
. . . . Q . . .
Above configuration has the fitness: (1.0,)
Q . . . . . . .
. . . Q . . . .
. . . . . . . Q
. . Q . . . . .
. . . . Q . . .
. . . . . . Q .
. Q . . . . . .
. . . . . Q . .
Above configuration has the fitness: (1.0,)
. . . . . Q . .
. Q . . . . . .
. . . . Q . . .
Q . . . . . . .
. . . . . . Q .
. . . Q . . . .
. . . . . . . Q
. . Q . . . . .
Above config

# Exerciții

1. Cum funcționează operatorii [`cxPartialyMatched`](https://www.hindawi.com/journals/cin/2017/7430125/) și [`mutShuffleIndexes`](https://deap.readthedocs.io/en/master/api/tools.html#deap.tools.mutShuffleIndexes) folosiți în exemplu?

2. Ce operatori de încrucișare ar fi încă compatibile cu reprezentarea de tip permutație?

3. În configurația curentă a metodei, până la ce număr de dame putem obține soluții optime în mod stabil (metoda găsește măcar o soluție optimă la fiecare 2-3 rulări)? 

4. Cu ce mărime de populație putem găsi soluțiile optime și penttru versiunea de problemă cu 32 de dame?

5. Modificați rezolvarea problemei, trecând de la o reprezentare de tip permutație, la una de tip listă de coordonate  (`row`, `col` pt. fiecare piesă), unde algoritmul evolutiv va determina rândul și coloana pentru fiecare piesă (nu există  coordonate prestabilite). Cum afectează procesul de căutare, convergența metodei această reprezentare impropice?



csParialyMatched functioneaza ca operator de crossover pentru 2 parinti.
folosind random cu 2 cutpoints(indexul cutpoint) - folosit ca separator

Exemple:

   P1 = (3 4 8 | 2 7 1 | 6 5)
   --------------------------
   P2 = (4 2 5 | 1 6 8 | 3 7)
   --------------------------
cutpointul este " | " 
mapare este facuta cu 2 cutpoints


  P1 = (X X X | 1 6 8 | X X)
  --------------------------
  P2 = (X X X | 2 7 1 | X X) 
  --------------------------
mapping 1 <> 2, 6<>7 si 8<>1
input missing bits ( intre 2 cutpoints )


  P1 = (3 4 X | 1 6 8 | X 5)
  --------------------------
  P2 = (4 X 5 | 2 7 1 | 3 X) 
  --------------------------
Adaugare elemente lipsa. 
P1: se verifica valorile lipsa pentru elementul initial si se verifica ca ar trebui sa fie pus valoarea 8 dar aceasta deja exista. Prin urmare, se verifica maparea pentru 8, ar trebui sa punem valoarea 1 dar si aceasta exista. Se mai verifica o data si se vede ca pentru 1 punem valoarea 2. Cum 2 nu exista in sir => o valoare valida. Se repeta procesul pentru tot sirul si se trece la urmatorul, P2.


Varianta finala va fi:

  P1 = (3 4 2 | 1 6 8 | 7 5)
  --------------------------
  P2 = (4 8 5 | 2 7 1 | 3 6) 
  --------------------------